## Capstone Project - Battle of Neighbour hood
### Week 3
### Author : Umeshpratap Singh
#### This project is final journey towards IBM -Data Science Certification
It is a capstone proejct to demonstrate my learing in various data science subjects using python learned over course of IBM data science certification


In [1]:


import pandas as pd
import numpy as np


## Part 1 ( week 3) : Data Wrangling from Wikipedia for Toronto 

In [2]:
#  install BeautiulSoup  
 ! pip install requests BeautifulSoup4  # only once


In [3]:
# import library 
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [4]:
# function to get URL and point out issue if any 
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    


In [5]:
# creating object
raw_html = simple_get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
len(raw_html)


56520

In [6]:
# using beautifulSoup to convert web site table in to dataframe
soup = BeautifulSoup(raw_html, 'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'}) # required table on webiste
table_rows = table.find_all('tr')


res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df_toronto = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])

df_toronto.head()



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# removing not assigined value in Borough
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.head(5)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# to check how many Neighbourhood has not assigned value 
dftest = df_toronto[df_toronto.Neighbourhood == 'Not assigned']
dftest.head()

,PostalCode,Borough,Neighbourhood


#### There are no Neighbourhood which are not assigned 

In [9]:
df_toronto.groupby(['PostalCode']).count()

,Borough,Neighbourhood
PostalCode,,
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1
M1J,1,1
M1K,1,1
M1L,1,1
M1M,1,1


#### there are no postal code which is repeated twice and hence no merging of rows requried 

In [10]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df_toronto.shape # final shape of Data frame 

(103, 3)

### Part II of Week 3 assignment , Finding lat and long of each postal code

In [12]:
# Download file 
!wget -q -O 'toronto_latlon.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
latlan_df = pd.read_csv('toronto_latlon.csv')
latlan_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_ftoronto = pd.merge(df_toronto, latlan_df, left_on='PostalCode', right_on='Postal Code')
df_ftoronto.drop(["Postal Code"], axis = 1, inplace = True) # drop the second postal code

df_ftoronto.head() # final toronto DF 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Above is final result required 

In [16]:
df_ftoronto.shape # cross checking if all rows are there 

(103, 5)

### part 3 of week 3 : mapping on toronto with foursquare data

In [18]:
#import required libiaries 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011
Libraries imported.


In [19]:
# find lat long for Toronto , to map 
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating map of Toronto

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ftoronto['Latitude'], df_ftoronto['Longitude'], df_ftoronto['Borough'], df_ftoronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
# for visulaisation only including downtown toronto

downtown_data = df_ftoronto[df_ftoronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.shape

(19, 5)

In [22]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [45]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [23]:
#my credential
CLIENT_ID = 'LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO' # your Foursquare ID
CLIENT_SECRET = 'MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO
CLIENT_SECRET:MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX


#### Exlporing one of Neighbourhood


In [24]:
downtown_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [25]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Getting 100 venues near Regent Park

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=LDBN1VLX5GUKN31ABNLDPRLDYGQG0ZFR51AVIYI2QQOVJQMO&client_secret=MS4UUFERGH5VCOQUARZEJWA4TVWPI51NRN2BTUYMRVD4TMOX&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [27]:
# checking with four square and getting the list 
results = requests.get(url).json()


In [28]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### borrowing function from foursquare

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

#### Cleaning the json and creating dataframe 

In [32]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.653447,-79.362017
1,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.653559,-79.361809
2,Cooper Koo Family YMCA,"[{'id': '52e81612bcbc57f1066b7a37', 'name': 'D...",43.653249,-79.358008
3,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",43.654735,-79.359874
4,Impact Kitchen,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",43.656369,-79.356980


In [33]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [36]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.9.final.0
python-bits: 64
OS: Linux
OS-release: 4.4.0-184-generic
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: en_US.UTF-8
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.24.1
pytest: 4.2.1
pip: 19.1.1
setuptools: 40.8.0
Cython: 0.29.5
numpy: 1.15.4
scipy: 1.2.0
pyarrow: 0.15.1
xarray: None
IPython: 7.2.0
sphinx: 1.8.4
patsy: 0.5.1
dateutil: 2.7.5
pytz: 2018.9
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.9
feather: None
matplotlib: 3.0.2
openpyxl: 2.6.0
xlrd: 1.2.0
xlwt: 1.3.0
xlsxwriter: 1.1.2
lxml.etree: 4.3.1
bs4: 4.7.1
html5lib: 1.0.1
sqlalchemy: 1.2.18
pymysql: None
psycopg2: 2.7.6.1 (dt dec pq3 ext lo64)
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
gcsfs: None


In [37]:
print(pd.__version__)


0.24.1
